In [2]:
#import os

#os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import jax
import jax.numpy as jnp
from jax import random, vmap
from flax import linen as nn
import optax

# Neural network

In [3]:
batch_size = 8
input_dim = 768
x = random.normal(key=random.PRNGKey(42), shape=(batch_size, input_dim))
y = random.normal(key=random.PRNGKey(42), shape=(batch_size, 1))

In [4]:
print(x.shape, y.shape)

(8, 768) (8, 1)


In [5]:
class JaxNet(nn.Module):
    def setup(self):
        self.W1 = nn.Dense(features=64)
        self.W2 = nn.Dense(features=1)
    
    def __call__(self, x):
        x = self.W1(x)
        x = nn.relu(x)
        x = self.W2(x)
        return x
    
def mse_loss(params, model, x, y):
    pred = model.apply(params, x)
    return (pred - y) ** 2


In [6]:
model = JaxNet()
params = model.init(random.PRNGKey(42), x[0])

optimizer = optax.adam(learning_rate=3e-4)
opt_state = optimizer.init(params)

# Using Jacfwd on layers

In [29]:
params

{'params': {'W1': {'kernel': Array([[-0.00463279,  0.01600334,  0.05311706, ..., -0.06047693,
            0.01630949, -0.04598638],
          [ 0.00522775,  0.00520009,  0.05464223, ..., -0.07799789,
            0.02453554, -0.00683571],
          [-0.00268752,  0.02008145,  0.0627301 , ..., -0.01019854,
           -0.01330569,  0.00713905],
          ...,
          [ 0.06569487,  0.02110667,  0.04668283, ...,  0.00330125,
           -0.01525788,  0.0039668 ],
          [ 0.06391574, -0.01504249, -0.01096328, ..., -0.02364219,
           -0.06919332, -0.01007098],
          [-0.01518737,  0.01303389,  0.05356703, ..., -0.04577949,
            0.01352515,  0.07838021]], dtype=float32),
   'bias': Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], d

In [7]:
xi, yi = x[0], y[0]

gradients = jax.jit(jax.jacfwd(mse_loss, argnums=0)(params, model, xi, yi))

#gradients = jax.tree_util.tree_map( lambda w: jax.jacfwd(mse_loss, argnums=1)(w, params, model, xi, yi), params )

2023-09-15 23:45:12.650807: W external/tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 9.05GiB (rounded to 9714468096)requested by op 
2023-09-15 23:45:12.651123: W external/tsl/tsl/framework/bfc_allocator.cc:497] *********************************************************__________________________________________*
2023-09-15 23:45:12.651241: E external/xla/xla/pjrt/pjrt_stream_executor_client.cc:2593] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 9714467844 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:    9.05GiB
              constant allocation:         0B
        maybe_live_out allocation:    9.05GiB
     preallocated temp allocation:         0B
                 total allocation:   18.09GiB
              total fragmentation:         0B (0.00%)
Peak buffers:
	Buffer 1:
		Size: 9.05GiB
		Entry Parameter Subshape: s32[49281,49281]

	Buffer 2:

XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 9714467844 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:    9.05GiB
              constant allocation:         0B
        maybe_live_out allocation:    9.05GiB
     preallocated temp allocation:         0B
                 total allocation:   18.09GiB
              total fragmentation:         0B (0.00%)
Peak buffers:
	Buffer 1:
		Size: 9.05GiB
		Entry Parameter Subshape: s32[49281,49281]
		==========================

	Buffer 2:
		Size: 9.05GiB
		Operator: op_name="jit(add)/jit(main)/add" source_file="/tmp/ipykernel_35158/440665117.py" source_line=3
		XLA Label: fusion
		Shape: s32[49281,49281]
		==========================

	Buffer 3:
		Size: 4B
		Entry Parameter Subshape: s32[]
		==========================



In [24]:
gradients

{'params': {'W1': {'bias': Array([[-0.6662318 , -0.        , -0.        , -0.        ,  0.36837235,
           -0.        , -0.        , -0.2865735 , -0.        ,  0.9268758 ]],      dtype=float32),
   'kernel': Array([[[-0.0382093 , -0.        , -0.        , -0.        ,
             0.02112666, -0.        , -0.        , -0.01643538,
            -0.        ,  0.0531576 ],
           [-0.23542339, -0.        , -0.        , -0.        ,
             0.1301701 , -0.        , -0.        , -0.10126521,
            -0.        ,  0.327526  ],
           [ 0.02539205, -0.        , -0.        , -0.        ,
            -0.01403975, -0.        , -0.        ,  0.01092216,
            -0.        , -0.03532596],
           [-0.4762796 , -0.        , -0.        , -0.        ,
             0.2633441 , -0.        , -0.        , -0.2048673 ,
            -0.        ,  0.6626103 ],
           [-0.31153318, -0.        , -0.        , -0.        ,
             0.17225267, -0.        , -0.        , -0.13400

In [25]:
loss_jacobian = jax.jacfwd(mse_loss, argnums=0)(params, model, evaluation_point, evaluation_target)
loss_jacobian

{'params': {'W1': {'bias': Array([[-8.955847  , -0.        , -0.        , -0.02139482,  4.9518595 ,
            7.8320136 , -6.778864  , -3.852275  , -0.        , 12.459564  ]],      dtype=float32),
   'kernel': Array([[[-8.9558468e+00, -0.0000000e+00, -0.0000000e+00, -2.1394815e-02,
             4.9518595e+00,  7.8320136e+00, -6.7788639e+00, -3.8522749e+00,
            -0.0000000e+00,  1.2459564e+01],
           [-4.4779234e+00, -0.0000000e+00, -0.0000000e+00, -1.0697408e-02,
             2.4759297e+00,  3.9160068e+00, -3.3894320e+00, -1.9261374e+00,
            -0.0000000e+00,  6.2297821e+00],
           [-1.3433770e+01, -0.0000000e+00, -0.0000000e+00, -3.2092221e-02,
             7.4277892e+00,  1.1748021e+01, -1.0168296e+01, -5.7784123e+00,
            -0.0000000e+00,  1.8689344e+01],
           [-1.7911694e+01, -0.0000000e+00, -0.0000000e+00, -4.2789631e-02,
             9.9037189e+00,  1.5664027e+01, -1.3557728e+01, -7.7045498e+00,
            -0.0000000e+00,  2.4919128e+01],
   

In [26]:
loss_jacobian["params"]["W1"]["kernel"].shape

(1, 5, 10)

In [27]:
gradients = jax.tree_map(lambda x: jnp.mean(x, axis=(0,1)), loss_jacobian)
gradients

{'params': {'W1': {'bias': Array(0.56350565, dtype=float32),
   'kernel': Array([-10.747017  ,   0.        ,   0.        ,  -0.02567378,
            5.942231  ,   9.3984165 ,  -8.134636  ,  -4.6227303 ,
            0.        ,  14.951477  ], dtype=float32)},
  'W2': {'bias': Array(-21.42035, dtype=float32),
   'kernel': Array([-8.777163], dtype=float32)}}}

In [28]:
gradients["params"]["W1"]["kernel"].shape

(10,)

In [29]:
loss_jacobian = vmap(jax.jacfwd(mse_loss, argnums=0), in_axes=(None, None, 0, 0), out_axes=0)(params, model, x, y)
loss_jacobian

{'params': {'W1': {'bias': Array([[[-0.66623163, -0.        , -0.        , -0.        ,
             0.36837226, -0.        , -0.        , -0.28657338,
            -0.        ,  0.9268756 ]],
   
          [[-0.        , -0.05249475, -0.        , -0.        ,
             0.02165244, -0.        , -0.02964118, -0.01684441,
             0.00816008, -0.        ]],
   
          [[ 0.        ,  0.        ,  0.        ,  0.        ,
            -0.26315907,  0.        ,  0.36025247,  0.20472331,
            -0.09917583,  0.        ]],
   
          [[ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        , -0.46606568,  0.        ,  0.2292403 ,
             0.        ,  0.        ]],
   
          [[-0.        , -0.        , -0.        , -0.        ,
            -0.        ,  0.47976726, -0.        , -0.23597959,
            -0.        , -0.        ]],
   
          [[ 0.        ,  0.        ,  0.        ,  0.        ,
             0.        , -0.24109204,  0.        ,  

In [30]:
loss_jacobian["params"]["W2"]["kernel"].mean((0,1,2))

Array([0.1828678], dtype=float32)

In [31]:
gradients = jax.tree_map(lambda x: jnp.sum(x, axis=(0, 1,2)), loss_jacobian)
gradients

{'params': {'W1': {'bias': Array(-5.321797, dtype=float32),
   'kernel': Array([-0.7296927 , -0.05353004, -0.1413371 ,  0.00319662, -0.3916064 ,
          -1.7406904 ,  1.3203006 ,  1.0290506 , -0.36347264,  1.0151641 ],      dtype=float32)},
  'W2': {'bias': Array(7.949466, dtype=float32),
   'kernel': Array([14.629423], dtype=float32)}}}

In [32]:
updates, opt_state = optimizer.update(loss_jacobian, opt_state)

In [33]:
params = optax.apply_updates(params, updates)
params

{'params': {'W1': {'bias': Array([[[ 0.0003,  0.    ,  0.    ,  0.    , -0.0003,  0.    ,  0.    ,
             0.0003,  0.    , -0.0003]],
   
          [[ 0.    ,  0.0003,  0.    ,  0.    , -0.0003,  0.    ,  0.0003,
             0.0003, -0.0003,  0.    ]],
   
          [[ 0.    ,  0.    ,  0.    ,  0.    ,  0.0003,  0.    , -0.0003,
            -0.0003,  0.0003,  0.    ]],
   
          [[ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.0003,  0.    ,
            -0.0003,  0.    ,  0.    ]],
   
          [[ 0.    ,  0.    ,  0.    ,  0.    ,  0.    , -0.0003,  0.    ,
             0.0003,  0.    ,  0.    ]],
   
          [[ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.0003,  0.    ,
            -0.0003,  0.    ,  0.    ]],
   
          [[-0.0003,  0.    ,  0.0003,  0.    ,  0.    ,  0.0003,  0.    ,
             0.    ,  0.    ,  0.0003]],
   
          [[-0.0003,  0.    ,  0.0003, -0.0003,  0.    ,  0.0003,  0.    ,
             0.    ,  0.    ,  0.0003]]], dtype=float32),
   

In [34]:
gradients

{'params': {'W1': {'bias': Array(-5.321797, dtype=float32),
   'kernel': Array([-0.7296927 , -0.05353004, -0.1413371 ,  0.00319662, -0.3916064 ,
          -1.7406904 ,  1.3203006 ,  1.0290506 , -0.36347264,  1.0151641 ],      dtype=float32)},
  'W2': {'bias': Array(7.949466, dtype=float32),
   'kernel': Array([14.629423], dtype=float32)}}}

In [35]:
model2 = JaxNet()
params2 = model.init(random.PRNGKey(42), x[0])

optimizer_def = optax.adam(learning_rate=1e-3)
optimizer = optimizer_def.init(params2)

def loss_fn(params2, x, y):
    preds = model.apply(params2, x)
    loss = jnp.mean((preds - y)**2)
    return loss

grad_fn = jax.jit(jax.grad(loss_fn))

grads = grad_fn(params2, x, y)

In [36]:
grads

{'params': {'W1': {'bias': Array([ 0.32835847, -0.00656184, -0.18990612,  0.00065913,  0.0158582 ,
          -0.3884065 ,  0.04132641,  0.0016438 , -0.01137697, -0.45681924],      dtype=float32),
   'kernel': Array([[ 3.25176507e-01,  9.02128522e-04, -1.52221411e-01,
            3.59859027e-04, -3.63957956e-02, -3.88265491e-01,
            5.34393154e-02,  7.73611218e-02, -1.47115951e-02,
           -4.52392459e-01],
          [-3.29341173e-01, -4.55594389e-03,  1.38362929e-01,
           -2.10456201e-04,  1.31507935e-02,  2.64409125e-01,
            4.27177083e-03, -1.12786386e-02, -1.17599871e-03,
            4.58186418e-01],
          [-2.92675406e-01, -4.65021748e-03,  1.36488110e-01,
           -1.96029345e-04, -6.05238229e-02,  2.35373229e-01,
            8.04518089e-02,  5.85698113e-02, -2.21480075e-02,
            4.07176226e-01],
          [ 2.68808454e-01,  2.97672022e-03, -1.51478976e-01,
            7.33235036e-04,  3.22636627e-02, -3.94895792e-01,
            8.95771838e-0

In [37]:
updates, optimizer_state = optimizer.update(grads, optimizer.state)
updates

AttributeError: 'tuple' object has no attribute 'update'

In [ ]:
gradients["params"]["W2"]["kernel"] == params2["params"]["W2"]["kernel"]